# Multiple materials
This example shows how to use two different materials on a
grid with the same cells everywhere. Hence, the only difference
is the type of material and the state variables.
We start by including the necessary packages, as well as the
[`J2Plasticity.jl`](J2Plasticity.jl) and
[`MaterialModelsBaseElement.jl`](MaterialModelsBaseElement.jl)
files from the previous example.

In [1]:
using Tensors, MaterialModelsBase, Ferrite, FerriteAssembly
include("J2Plasticity.jl");
include("MaterialModelsBaseElement.jl");

Then, we also define an elastic material

In [2]:
struct ElasticMaterial{T<:SymmetricTensor{4,3}} <: AbstractMaterial
    D::T
end
function ElasticMaterial(;E, ν)
    D, _, _ = elastic_conversion(E, ν)
    return ElasticMaterial(D)
end;

This material requires not state, so `MaterialModelsBase.jl`'s `NoMaterialState`
will be created by default. Hence, we only need to define the `material_response`

In [3]:
function MaterialModelsBase.material_response(
    material::ElasticMaterial, ϵ::SymmetricTensor{2,3}, state,
    Δt, cache=get_cache(material), args...; kwargs...)
    σ = material.D ⊡ ϵ
    return σ, material.D, NoMaterialState()
end;

The same `element_routine!` as before can be used, defined in
[`MaterialModelsBaseElement.jl`](MaterialModelsBaseElement.jl).
Hence, we just need to define the materials, grid, cellvalues etc.

In [4]:
material_pl = J2Plasticity(200.0e9, 0.3, 200.0e6, 10.0e9);
material_el = ElasticMaterial(E=200.0e9, ν=0.3);
grid = generate_grid(Tetrahedron, (20,2,4), zero(Vec{3}), Vec((10.0,1.0,1.0)));
cellvalues = CellVectorValues(
    QuadratureRule{3,RefTetrahedron}(2), Lagrange{3, RefTetrahedron, 1}());
dh = DofHandler(grid); push!(dh, :u, 3); close!(dh); # Create dofhandler
K = create_sparsity_pattern(dh);
r = zeros(ndofs(dh));

We then define the cellsets where we have each material

In [5]:
addcellset!(grid, "elastic", x -> x[1] <= 5.0+eps())
set_el = getcellset(grid, "elastic");
addcellset!(grid, "plastic", setdiff(1:getncells(grid), set_el));
set_pl = getcellset(grid, "plastic");

The key to handling multiple materials, with different state variables,
using the regular DofHandler or a single FieldHandler (in the MixedDofHandler case)
is to create multiple state variables, and call `doassemble!` for each of them:

In [6]:
states_el = create_states(dh, _->initial_material_state(material_el), cellvalues, set_el);
states_pl = create_states(dh, _->initial_material_state(material_pl), cellvalues, set_pl);

We also need buffers for each material

In [7]:
buffer_el = CellBuffer(dh, cellvalues, material_el, nothing, get_cache(material_el));
buffer_pl = CellBuffer(dh, cellvalues, material_pl, nothing, get_cache(material_pl));

And then we define the displacements and the assembler, before assembling

In [8]:
a = zeros(ndofs(dh))
assembler = start_assemble(K, r)
doassemble!(assembler, buffer_el, states_el, dh, a; cellset=set_el);
doassemble!(assembler, buffer_pl, states_pl, dh, a; cellset=set_pl);

using Test #hide
K_ref = create_sparsity_pattern(dh); #hide
r_ref = zeros(ndofs(dh)); #hide
states = create_states(dh, _->initial_material_state(material_el), cellvalues); #hide
a_ref = zeros(ndofs(dh)) #hide
assembler_ref = start_assemble(K_ref,r_ref) #hide
doassemble!(assembler_ref, buffer_el, states, dh, a_ref); #hide
@test K ≈ K_ref; #hide

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*